## Importing libraries

In [19]:
from math import ceil
from typing import Dict, List, Optional, Union

import cirq
from IPython.display import display, Math, Latex, HTML
import numpy as np

In [24]:
def vector_representation(array):
    res = r"\begin{pmatrix}" + str(array[0])
    for x in array[1:]:
        res += r"\\" + str(x)
    return res + r"\end{pmatrix}"

## Creating classical data

In [2]:
is_low_rank_assumption_passed = False
N_TRIES_LIMIT = 10

i = 0
while not is_low_rank_assumption_passed and i <= N_TRIES_LIMIT:
    i += 1
    # Number of users
    m = 16
    # Number of products
    n = 8
    # Preference matrix
    P = np.random.randint(2, size=(m, n))
    
    # Rank of approximation
    k = 5
    
    U, S, V = np.linalg.svd(P)
    S[k:] = 0
    rank = np.sum(S > 0)
    is_low_rank_assumption_passed = rank == k
    if is_low_rank_assumption_passed and [0, 0, 0, 0, 0, 0, 0, 0] not in P:
        reconstruct_S = np.zeros((U.shape[0], V.shape[0]))
        np.fill_diagonal(reconstruct_S, S)
        approximation_matrix = U @ reconstruct_S @ V
        epsilon = np.linalg.norm(P - approximation_matrix) / np.linalg.norm(P)

if i == N_TRIES_LIMIT + 1:
    raise RuntimeError("Couldn't find an approximation matrix with the given parameters.")

print(f"P = {P}")

P = [[0 1 0 0 1 0 1 1]
 [1 0 1 1 0 0 0 1]
 [0 1 1 1 1 1 0 0]
 [0 1 0 1 1 1 0 1]
 [0 1 1 0 0 0 1 1]
 [1 0 1 1 1 0 1 0]
 [0 0 1 0 1 0 0 1]
 [1 0 0 1 0 1 1 1]
 [1 0 0 1 0 1 1 0]
 [1 1 0 0 1 1 1 1]
 [0 0 1 0 1 0 0 0]
 [0 1 1 1 0 1 0 1]
 [0 0 0 0 1 1 0 0]
 [1 0 0 0 0 1 1 0]
 [1 0 0 1 1 1 1 0]
 [0 1 0 1 1 0 1 1]]


## Creating data structure

In [3]:
class _Leaf:
    def __init__(self, value: float, positive_sign: bool):
        self.value: float = value
        self.positive_sign: bool = positive_sign
    
    @property
    def is_leaf(self):
        return True

class _BinaryTree:
    def __init__(self, depth: int, value: float = 0.):
        self.value: float = value
        self.depth: int = depth
        self.left: Union[_BinaryTree, _Leaf] = _Leaf(0, True)
        self.right: Union[_BinaryTree, _Leaf] = _Leaf(0, True)
    
    def __getitem__(self, key: int):      
        if key - 1 < 0:
            raise KeyError(f"Invalid key: {key} (must be strictly positive).")
            
        binary_path: str = bin(key - 1)[2:]
        binary_path = '0' * (self.depth - len(binary_path)) + binary_path
            
        if len(binary_path) != self.depth:
            raise KeyError(f"Invalid key for storage: {position} (too long).")
        
        def get_aux(binary_tree: _BinaryTree, binary_path: str):
            if len(binary_path) == 0:
                raise ValueError(f"Invalid value encountered for binary_path: nil length.")
            
            position: str = binary_path[0]
            
            if len(binary_path) == 1:
                if position == "0":
                    if binary_tree.left.value == -1:
                        raise KeyError()
                    return binary_tree.left.value, binary_tree.left.positive_sign
                if binary_tree.right.value == -1:
                    raise KeyError()
                return binary_tree.right.value, binary_tree.right.positive_sign

            if position == "0":
                if isinstance(binary_tree.left, _Leaf):
                    raise KeyError()
                
                return get_aux(binary_tree.left, binary_path[1:])
            else:
                if isinstance(binary_tree.right, _Leaf):
                    raise KeyError()
                
                return get_aux(binary_tree.right, binary_path[1:])
        
        try:
            return get_aux(self, binary_path)
        except KeyError:
            raise KeyError(f"No value stored for key: {key}.")
    
    def __str__(self):
        return f"_BinaryTree(depth={self.depth}, value={self.value:.2f})"
    
    @property
    def is_leaf(self) -> bool:
        return False
    
    def store(self, position: int, value: float, positive_sign: bool) -> None:
        def store_aux(binary_tree: _BinaryTree, binary_path: str, value: float, positive_sign: bool) -> None:
            if len(binary_path) == 0:
                raise ValueError(f"Invalid value encountered for binary_path: nil length.")
            
            position: str = binary_path[0]
            binary_tree.value += value
            
            if len(binary_path) == 1:
                if position == "0":
                    binary_tree.left = _Leaf(value, positive_sign)
                else:
                    binary_tree.right = _Leaf(value, positive_sign)
            else:
                if position == "0":
                    if binary_tree.left.is_leaf:
                        binary_tree.left = _BinaryTree(len(binary_path) - 1)
                        
                    store_aux(binary_tree.left, binary_path[1:], value, positive_sign)
                else:
                    if binary_tree.right.is_leaf:
                        binary_tree.right = _BinaryTree(len(binary_path) - 1)
                        
                    store_aux(binary_tree.right, binary_path[1:], value, positive_sign)
        
        if isinstance(position, int):
            if position - 1 < 0:
                raise ValueError(f"Invalid position: {position} (must be strictly positive).")
                
            binary_path = bin(position - 1)[2:]
            binary_path = '0' * (self.depth - len(binary_path)) + binary_path
            
            if len(binary_path) != self.depth:
                raise KeyError(f"Invalid key for storage: {position} (too long).")
            
            store_aux(self, binary_path, value, positive_sign)
        else:
            raise TypeError(f"Incorrect type for position: {type(position)} (int expected).")
            

class QRAM:
    def __init__(self, array: np.ndarray):
        if not isinstance(array, np.ndarray):
            raise TypeError(f"Can only store np.ndarray ({type(array)} given).")
        if len(array.shape) > 2:
            raise ValueError(f"Can't store arrays with more than 2 dimensions (shape {array.shape} given).")
        if len(array.shape) == 1:
            array = array.reshape((1, array.shape[0]))

        self.m: int = array.shape[0]
        self.n: int = array.shape[1]
        n_binary_writing: str = bin(self.n)[3:]
        self.trees_depth: int = len(n_binary_writing) + 1 if "1" in n_binary_writing else len(n_binary_writing)
        m_binary_writing: str = bin(self.m)[3:]
        self.log2m: int = len(m_binary_writing) + 1 if "1" in m_binary_writing else len(m_binary_writing)
        self._trees: Dict[int, _BinaryTree] = {}
        norms: np.ndarray = np.linalg.norm(array, axis=1)
            
        if self.m == 1:
            self._store_init(array / norms)
            self.qram_norms: Optional[QRAM] = None
        else:
            self._store_init(array / norms.reshape((norms.shape[0], 1)))
            self.qram_norms: Optional[QRAM] = QRAM(norms)
            
    
    def __getitem__(self, key):
        if isinstance(key, int):
            return self._trees[key]
        if isinstance(key, tuple):
            if len(key) != 2:
                raise KeyError(f"An address is made of 2 indexes ({len(tuple)} given).")
            if self._trees.get(key[0]) is None:
                self._trees[i] = _BinaryTree(self.trees_depth)
                
            return self._trees[key[0]][key[1]]
        raise KeyError(f"Not a representable address: {key}.")
    
    def __repr__(self):
        return f"QRAM({[str(self._trees[x]) for x in self._trees]})"
    
    def __setitem__(self, key, value):
        if isinstance(key, tuple):
            if len(key) != 2:
                raise KeyError(f"An address is made of 2 indexes ({len(key)} given).")
            if not (isinstance(value, tuple) or isinstance(value, float) or isinstance(value, np.int64) or isinstance(value, np.float64)):
                raise TypeError(f"Not an acceptable type to store: {type(value)} (numeric type or tuple expected).")
            if isinstance(value, float) or isinstance(value, np.int64) or isinstance(value, np.float64):
                self._store(key[0], key[1], value ** 2, value >= 0)
                return
            if len(value) != 2:
                raise ValueError(f"value must be a tuple of length 2 (length {len(value)} given).")
            if not isinstance(value[0], float):
                raise ValueError(f"The first element of value must be a float ({type(value[0])} given).")
            if value[0] < 0 or value[0] > 1:
                raise ValueError(f"The first element of value must be between 0 and 1 ({value[0]} given).")
            if not isinstance(value[1], bool):
                raise ValueError(f"The second element of value must be a boolean ({type(value[1])} given).")
                
            self._store(key[0], key[1], value[0], value[1])
        else:
            raise TypeError(f"Invalid address given: {key}.")
    
    def _store(self, i: int, j: int, value: float, positive_sign: bool) -> None:
        if self._trees.get(i) is None:
            self._trees[i] = _BinaryTree(self.trees_depth)
        
        self._trees[i].store(j, value, positive_sign)
    
    def _store_init(self, array: np.ndarray) -> None:        
        for i in range(array.shape[0]):
            for j in range(array.shape[1]):
                self[i + 1, j + 1] = array[i, j]

## Creating gates

In [364]:
class _LoadingVectorPowGate(cirq.Gate):
    def __init__(self, qram: QRAM, index: int, exponent: int = 1.):
        self.qram: QRAM = qram
        self.index: int = index
        # Used to invert the gate
        if exponent not in [-1, 0, 1]:
            raise ValueError(f"_LoadingVectorGate can only be raised to power 1, 0 or -1 (exponent {exponent} given).")
        self.exponent: int = exponent
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.qram.trees_depth
    
    def _decompose_(self, qubits):
        def _decompose_aux(tree, qubits, binary_writing, operations):
            if not tree.value:
                return
            depth: int = len(binary_writing)
            theta: float = np.arccos(np.sqrt(tree.left.value / tree.value)) + np.arcsin(np.sqrt(tree.right.value / tree.value))
            operations.append((cirq.ry(theta) ** self.exponent).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing))
            
            if tree.left.is_leaf and tree.right.is_leaf:
                if tree.left.positive_sign and tree.right.positive_sign:
                    return
                elif tree.left.positive_sign:
                    operations.append((cirq.Z ** self.exponent).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing)) 
                    return
                elif tree.right.positive_sign:
                    operations.append((cirq.Z ** self.exponent).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing)) 
                    
                operations.append((cirq.ry(2 * np.pi) ** self.exponent).on(qubits[depth]).controlled_by(*qubits[:depth], control_values=binary_writing)) 
                return
            elif tree.left.is_leaf:
                _decompose_aux(tree.right, qubits, binary_writing + [1], operations)
                return
            elif tree.right.is_leaf:
                _decompose_aux(tree.left, qubits, binary_writing + [0], operations)
                return
            
            _decompose_aux(tree.left, qubits, binary_writing + [0], operations)
            _decompose_aux(tree.right, qubits, binary_writing + [1], operations)
        
        res = []
        _decompose_aux(self.qram._trees[self.index], qubits[:self.qram.trees_depth], [], res)
        
        return res if self.exponent >= 0 else res[::-1]
    
    def __pow__(self, exponent, modulo=None):
        return _LoadingVectorPowGate(self.qram, self.index, exponent)
    
    def __repr__(self):
        return f"LV({self.index}) ** {self.exponent}"

    
class LoadingVectorGate(_LoadingVectorPowGate):
    def __init__(self, qram: QRAM, index: int):
        _LoadingVectorPowGate.__init__(self, qram, index, 1.)
    
    def __repr__(self):
        return f"LV({self.index})"


class LoadingAngleGate(cirq.Gate):
    def __init__(self, index_size:int , precision: int, qram: QRAM, inverse: bool = False):
        self.index_size: int = index_size
        self.precision:int = precision
        self.qram: QRAM = qram
        self.inverse: bool = inverse
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.index_size + self.precision
    
    def _unitary_(self) -> np.ndarray:
        res = np.zeros((2 ** (self.index_size + self.precision), 2 ** (self.index_size + self.precision)))
        
        for x in range(2 ** self.index_size):
            for y in range(2 ** self.precision):
                binary_writing = bin(x)[2:]
                binary_writing = '0' * (self.index_size - len(binary_writing)) + binary_writing
                current = self.qram[1]
                
                for fig in binary_writing:
                    if fig == '0':
                        current = current.left
                    elif fig == '1':
                        current = current.right
                    else:
                        raise ValueError(f"Unknown figure : {fig}")
                
                if isinstance(current, _Leaf) or current.value == current.left.value:
                    angle = 0
                else:
                    angle = np.arccos(np.sqrt(current.left.value / current.value))
                
                f_value = [int(angle)] + ThresholdGate._bin_repr(angle - int(angle), self.precision - 1)
                y_value = bin(y)[2:]
                y_value = '0' * (self.precision - len(y_value)) + y_value
                y_value = [int(v) for v in y_value]
                quantum_state = [int(v) for v in binary_writing] + list(np.logical_xor(f_value, y_value).astype(int))
                index = 0
                
                for i, v in enumerate(quantum_state):
                    if v:
                        index += 2 ** (self.index_size + self.precision) // 2 ** (i + 1)
                    
                res[index, x * 2 ** self.precision + y] = 1
        
        return res.T if self.inverse else res
    
    def __pow__(self, exponent, modulo=None):
        if exponent == 1:
            return LoadingAngleGate(self.index_size, self.precision, self.qram, inverse=False)
        elif exponent == -1:
            return LoadingAngleGate(self.index_size, self.precision, self.qram, inverse=True)
        elif exponent == 0:
            return cirq.IdentityGate(num_qubits=self._num_qubits_())
        raise ValueError(f"Unknown exponent: {exponent}")
    
    def __repr__(self) -> str:
        return "LA ** -1" if self.inverse else "LA"
    

class _BasisChangeGate(cirq.Gate):
    def _num_qubits_(self) -> int:
        return 1
    
    def _unitary_(self) -> np.ndarray:
        return np.array([[1, -1 * 1j], [1 * 1j, -1]]) / np.sqrt(2)
    
    def __repr__(self) -> str:
        return "Y<->Z"


class ParallelizedLoadingVectorGate(cirq.Gate):
    def __init__(self, qram: QRAM, precision: int):
        self.qram: QRAM = qram
        self.precision: int = precision
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.qram.trees_depth + self.precision
    
    def _decompose_(self, qubits):
        operations = []
        to_rotate = qubits[:self.qram.trees_depth]
        ancillas = qubits[self.qram.trees_depth:]
        angle = np.arccos(np.sqrt(self.qram[1].left.value / self.qram[1].value))
        operations.append(cirq.ry(2 * angle).on(to_rotate[0]))
        
        for k in range(1, self.qram.trees_depth):
            operations.append(LoadingAngleGate(k, self.precision, self.qram).on(*(to_rotate[:k] + ancillas)))
            operations.append(_BasisChangeGate().on(to_rotate[k]))
            
#             for t in range(self.precision):
#                 operations.append(cirq.rz(2 ** (- t)).on(ancillas[t]).controlled_by(to_rotate[k]))
            
            operations.append(LoadingAngleGate(k, self.precision, self.qram, inverse=True).on(*(to_rotate[:k] + ancillas)))
            operations.append(_BasisChangeGate().on(to_rotate[k]))
        
        return operations
    
    def __repr__(self) -> str:
        return "PLVG"

class _LoadingPowGate(cirq.Gate):
    def __init__(self, qram: QRAM, exponent: int):
        self.qram: QRAM = qram
        
        if exponent not in [-1, 0, 1]:
            raise ValueError(f"_LoadingPowGate can only be raised to power 1, 0 or -1 (exponent {exponent} given).")
        
        self.exponent: int = exponent
    
    def _num_qubits_(self) -> int:
        return self.qram.trees_depth + self.qram.log2m
    
    def _decompose_(self, qubits):
        operations = []
        control_qubits, state_qubits = qubits[:self.qram.log2m], qubits[self.qram.log2m:]
        
        for i in range(2 ** self.qram.log2m):
            control_values = bin(i)[2:]
            control_values = "0" * (len(control_qubits) - len(control_values)) + control_values
            control_values = [int(c) for c in control_values]
            operations.append((LoadingVectorGate(self.qram, i + 1) ** self.exponent).on(*state_qubits).controlled_by(*control_qubits, control_values=control_values))
        
        return operations if self.exponent >= 0 else operations[::-1]
    
    def __pow__(self, exponent, modulo=None):
        return _LoadingPowGate(self.qram, exponent)
    
    def __repr__(self):
        return f"LG ** {self.exponent}"


class LoadingGate(_LoadingPowGate):
    def __init__(self, qram: QRAM):
        _LoadingPowGate.__init__(self, qram, 1)
    
    def __repr__(self):
        return "LG"


class NormGate(LoadingVectorGate):
    def __init__(self, qram: QRAM):
        LoadingVectorGate.__init__(self, qram.qram_norms, 1)
    
    def __repr__(self):
        return "N"

class _UVPowGate(cirq.Gate):
    def __init__(self, qram: QRAM, exponent: int):
        if not isinstance(exponent, int):
            raise TypeError(f"UVGate can't be raised to a non-integer exponent (exponent {exponent} given)")
        self.exponent: int = exponent
        self.qram: QRAM = qram
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.qram.trees_depth + self.qram.log2m
    
    def _decompose_(self, qubits):
        operations = []
        first_register, second_register = qubits[:self.qram.log2m], qubits[self.qram.log2m:]
        
        for _ in range(abs(exponent)):
            if exponent >= 0:
                # Apply V
                operations.append((NormGate(self.qram) ** - 1).on(*first_register))
                operations.append((cirq.IdentityGate() * -1).on(*second_register).controlled_by(*first_register, control_values=[0] * self.qram.log2m))
                operations.append(NormGate(self.qram).on(*first_register))
                
            # Apply U
            operations.append(LoadingGate(self.qram).on(*qubits) ** -1)
            operations.append((cirq.IdentityGate() * -1).on(*first_register).controlled_by(*second_register, control_values=[0] * self.qram.trees_depth))
            operations.append(LoadingGate(self.qram).on(*qubits))
            
            if exponent < 0:
                # Apply V
                operations.append((NormGate(self.qram) ** - 1).on(*first_register))
                operations.append((cirq.IdentityGate() * -1).on(*second_register).controlled_by(*first_register, control_values=[0] * self.qram.log2m))
                operations.append(NormGate(self.qram).on(*first_register))
        
        return operations
    
    def __repr__(self):
        return f"UV ** {self.exponent}"
    
    def __pow__(self, exponent, modulo=None):
        return _UVPowGate(self.qram, exponent)


class UVGate(_UVPowGate):
    def __init__(self, qram):
        _UVPowGate.__init__(self, qram, 1)
    
    def __repr__(self) -> str:
        return "UV"


class _QPEPowGate(cirq.Gate):
    def __init__(self, gate: cirq.Gate, epsilon: float, exponent: int):
        if exponent not in [-1, 0, 1]:
            raise ValueError(f"QPEGate can only be raised to power 1, 0 or -1 (exponent {exponent} given).")
        self.exponent: int = exponent
        self.gate: cirq.Gate = gate
        self.t: int = ceil(np.log2(1 / epsilon))
        self.probability: float = 1 - 1 / (2 * (2 ** additional_qubits - 2))
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return self.gate._num_qubits_() + self.t
    
    def _decompose_(self, qubits):
        operations = []
        first_register, second_register = qubits[:self.t], qubits[self.t:]
        operations.append(cirq.H.on_each(*first_register))
        operations += [self.gate.on(*second_register).controlled_by(first_register[i]) ** ((2 ** i) * self.exponent) for i in range(self.t)]
        operations.append(cirq.QFT(inverse=True).on(*first_register) ** self.exponent)
        
        return operations if self.exponent >= 0 else operations[::-1]
    
    def __pow__(self, exponent, modulo=None):
        return _QPEPowGate(self.gate, self.epsilon, exponent)
    
    def __repr__(self):
        return f"QPE({self.probability:.3f})"


class QPEGate(_QPEPowGate):
    def __init__(self, gate: cirq.Gate, epsilon: float):
        _QPEPowGate.__init__(self, gate, epsilon, 1)
    
    def __repr__(self) -> int:
        return "QPE"
    

class _QSVEPowGate(cirq.Gate):
    def __init__(qram_A: QRAM, epsilon: float, exponent: int):
        if exponent not in [-1, 0, 1]:
            raise ValueError(f"QSVEGate can only be raised to power 1, 0 or -1 (exponent {exponent} given).")
        self.exponent: int = exponent
        self.qram_A: QRAM = qram_A
        self.epsilon: float = epsilon
        self.qpe_gate: QPEGate = QPEGate(UVGate(self.qram_A), 2 * epsilon)
        cirq.Gate.__init__(self)
        
    def _num_qubits_(self) -> int:
        return self.qram_A.log2m + self.qram_A.trees_depth + self.qpe_gate.t
    
    def _decompose_(self, qubits):
        operations = []
        first_register = qubits[:self.qram_A.log2m]
        second_register = qubits[self.qram_A.log2m:self.qram_A.log2m + self.qram_A.trees_depth]
        third_register = qubits[self.qram_A.log2m + self.qram_A.trees_depth:]
        operations.append(NormGate(qram_A).on(*first_register) ** self.exponent)
        operations.append(self.qpe_gate.on(*qubits) ** self.exponent)
        operations.append((NormGate(qram_A) ** -1).on(*first_register) ** self.exponent)
        
        return operations if self.exponent >= 0 else operations[::-1]
    
    def __pow__(self, exponent, modulo=None):
        return _QSVEPowGate(self.qram_A, self.epsilon, exponent)
    
    def __repr__(self) -> str:
        return f"QSVE ** {self.exponent}"


class QSVEGate(_QSVEPowGate):
    def __init__(self, qram_A: QRAM, epsilon: float):
        _QSVEPowGate.__init__(self, qram_A, epsilon, 1)
    
    def __repr__(self) -> str:
        return "QSVE"

    
class ThresholdGate(cirq.Gate):
    def __init__(self, threshold: float, n_qubits: int):
        self.b1: List[int] = ThresholdGate._bin_repr(threshold, n_qubits)
        self.b2: List[int] = ThresholdGate._bin_repr(1 - threshold, n_qubits)
        self.n_qubits: int = n_qubits
        cirq.Gate.__init__(self)
    
    def _num_qubits_(self) -> int:
        return 2 * self.n_qubits + 2
    
    def _decompose_(self, qubits):
        operations = []
        a = qubits[:self.n_qubits]
        result_qubit = qubits[self.n_qubits]
        equal_ancilla = qubits[self.n_qubits + 1]
        res_found_ancilla = qubits[self.n_qubits + 2]
        ancillas = qubits[self.n_qubits + 3:]
        
        if self.b1[1]:
            operations.append(cirq.X.on(equal_ancilla).controlled_by(a[0], control_values=[0]))
        
        if self.b2[1]:
            operations.append(cirq.X.on(equal_ancilla).controlled_by(a[0]))
            
        operations.append(cirq.X.on(equal_ancilla).controlled_by(a[1]))
        operations.append(cirq.X.on(res_found_ancilla).controlled_by(equal_ancilla))
        operations.append(cirq.X.on(result_qubit).controlled_by(a[0], a[1], equal_ancilla, control_values=[0, 1, 1]))
        
        if self.b2[1]:
            operations.append(cirq.X.on(result_qubit).controlled_by(a[0], equal_ancilla))
        
        operations.append(cirq.X.on(equal_ancilla).controlled_by(a[1]))
        
        if self.b1[1]:
            operations.append(cirq.X.on(equal_ancilla).controlled_by(a[0], control_values=[0]))
        
        if self.b2[1]:
            operations.append(cirq.X.on(equal_ancilla).controlled_by(a[0]))
            
        for i in range(2, self.n_qubits):
            if self.b1[i]:
                operations.append(cirq.X.on(equal_ancilla).controlled_by(a[0], res_found_ancilla, control_values=[0, 0]))
        
            if self.b2[i]:
                operations.append(cirq.X.on(equal_ancilla).controlled_by(a[0], res_found_ancilla, control_values=[1, 0]))
            
            operations.append(cirq.X.on(equal_ancilla).controlled_by(a[i], res_found_ancilla, control_values=[1, 0]))
            operations.append(cirq.X.on(ancillas[i - 2]).controlled_by(equal_ancilla, res_found_ancilla, control_values=[0, 1]))
            operations.append(cirq.X.on(res_found_ancilla).controlled_by(equal_ancilla))
            operations.append(cirq.X.on(result_qubit).controlled_by(a[0], a[i], equal_ancilla, control_values=[0, 1, 1]))
            
            if self.b2[i]:
                operations.append(cirq.X.on(result_qubit).controlled_by(a[0], equal_ancilla))
            
            operations.append(cirq.X.on(equal_ancilla).controlled_by(a[i], ancillas[i - 2], control_values=[1, 0]))
            
            if self.b1[i]:
                operations.append(cirq.X.on(equal_ancilla).controlled_by(a[0], ancillas[i - 2], control_values=[0, 0]))
        
            if self.b2[i]:
                operations.append(cirq.X.on(equal_ancilla).controlled_by(a[0], ancillas[i - 2], control_values=[1, 0]))
        
        return operations
    
    def __repr__(self) -> str:
        return f"T_{self.b1}"
            
    
    @staticmethod
    def _bin_repr(num: float, n_bits: int) -> List[int]:
        """ Returns the best n_bits-bit approximation of num.
        """
        res = []
        temp = 0
        for i in range(1, n_bits + 1):
            if abs(num - (temp + 2 ** - i)) < abs(num - temp):
                res.append(1)
                temp += 2 ** - i
            elif abs(num - (temp - 2 ** - i)) < abs(num - temp):
                res.append(-1)
                temp -= 2 ** -i
            else:
                res.append(0)

        bin_ = bin(int(.5 + temp * 2 ** n_bits))[2:]
        bin_ = '0' * (n_bits - len(bin_)) + bin_

        return [int(x) for x in bin_]
        

## Testing LoadingVectorGate

In [365]:
N = 5
phi = np.random.random(N) - .4
phi /= np.linalg.norm(phi)

ds_phi = QRAM(phi)
circuit = cirq.Circuit()
qubits = [cirq.GridQubit(1, i) for i in range(ds_phi.trees_depth)]
circuit.append(LoadingVectorGate(ds_phi, 1).on(*qubits))
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
# print(circuit)
visualization_circuit = cirq.Circuit()
visualization_circuit.append(cirq.decompose(circuit[0].operations[0]))
# print('\n' + '=' * 115 + '\n')
print(visualization_circuit)
to_display = r"\varphi=" + vector_representation(np.hstack((phi, np.zeros(result.final_state.shape[0] - N))))
to_display += r"\;|\varphi\rangle=" + vector_representation(np.real(result.final_state))
to_display += r"\,\|\varphi-|\varphi\rangle\|={}".format(np.linalg.norm(np.hstack((phi, np.zeros(result.final_state.shape[0] - N))) / np.linalg.norm(phi) - result.final_state))
display(Math(to_display))

(1, 0): ───Ry(0.221π)───(0)──────────(0)─────────(0)───(0)──────────@──────────@──────────@─────@──────────
                        │            │           │     │            │          │          │     │
(1, 1): ────────────────Ry(0.618π)───(0)─────────(0)───@────────────Ry(0.0π)───(0)────────(0)───(0)────────
                                     │           │     │                       │          │     │
(1, 2): ─────────────────────────────Ry(0.08π)───Z─────Ry(0.149π)──────────────Ry(0.0π)───Z─────Ry(2.0π)───


<IPython.core.display.Math object>

## Testing ParallelizedLoadingVectorGate

In [366]:
from cirq.circuits import InsertStrategy

N = 3
# phi = np.random.random(N) - .4
phi = np.array([1, 0, 0, 0])
# phi /= np.linalg.norm(phi)
precision = 5

ds_phi = QRAM(phi)
circuit = cirq.Circuit()
qubits = [cirq.LineQubit(i) for i in range(ds_phi.trees_depth + precision)]
circuit.append(ParallelizedLoadingVectorGate(ds_phi, precision).on(*qubits))
circuit.append(_BasisChangeGate().on(qubits[1]))
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
# print(circuit)
visualization_circuit = cirq.Circuit()
visualization_circuit.append(cirq.decompose(circuit[0].operations[0]))
# visualization_circuit.append(_BasisChangeGate().on(qubits[1]))
# # print('\n' + '=' * 115 + '\n')
print(visualization_circuit)
# to_display = r"\varphi=" + vector_representation(np.hstack((phi, np.zeros(result.final_state.shape[0] - N))))
# to_display += r"\;|\varphi\rangle=" + vector_representation(np.real(result.final_state))
# to_display += r"\,\|\varphi-|\varphi\rangle\|={}".format(np.linalg.norm(np.hstack((phi, np.zeros(result.final_state.shape[0] - N))) / np.linalg.norm(phi) - result.final_state))
# display(Math(to_display))
phi, result.final_state[::2 ** precision]

                 ┌───────┐
0: ───Ry(0.0π)────LA─────────LA ** -1───
                  │          │
1: ───Y<->Z───────┼─Y<->Z────┼──────────
                  │          │
2: ───────────────#2─────────#2─────────
                  │          │
3: ───────────────#3─────────#3─────────
                  │          │
4: ───────────────#4─────────#4─────────
                  │          │
5: ───────────────#5─────────#5─────────
                  │          │
6: ───────────────#6─────────#6─────────
                 └───────┘


(array([1, 0, 0, 0]),
 array([0.7071067+0.j       , 0.       +0.7071067j, 0.       +0.j       ,
        0.       +0.j       ], dtype=complex64))

In [368]:
(cirq.unitary(LoadingAngleGate(1, precision, ds_phi)) == np.eye(64)).all()

True

In [301]:
f = LoadingAngleGate(1, 2, ds_phi)
cirq.unitary(f)

0 0 [0, 1] 0.8615645720560159 0.46295740652254896 0.7480183636636442
0 1 [0, 1] 0.8615645720560159 0.46295740652254896 0.7480183636636442
0 2 [0, 1] 0.8615645720560159 0.46295740652254896 0.7480183636636442
0 3 [0, 1] 0.8615645720560159 0.46295740652254896 0.7480183636636442
1 0 [0, 0] 0.1384354279439843 0.1384354279439843 0
1 1 [0, 0] 0.1384354279439843 0.1384354279439843 0
1 2 [0, 0] 0.1384354279439843 0.1384354279439843 0
1 3 [0, 0] 0.1384354279439843 0.1384354279439843 0


array([[0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [307]:
x = np.arccos(np.sqrt(ds_phi[1].left.left.value/ds_phi[1].left.value))
x, [int(x)] + ThresholdGate._bin_repr(x - int(x), 1)

(0.7480183636636442, [0, 1])

In [305]:
ds_phi[1].left.left.value

0.46295740652254896

## Testing LoadingGate

In [78]:
circuit = cirq.Circuit()
qram_P = QRAM(P)
qubits = [cirq.GridQubit(0, i) for i in range(qram_P.log2m + qram_P.trees_depth)]
circuit.append(cirq.H.on_each(*qubits[:4]))
circuit.append(LoadingGate(qram_P).on(*qubits))
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
print(np.real(result.final_state).reshape(P.shape))
print(P / np.linalg.norm(P, axis=1).reshape(16, 1))
print(circuit)

[[0.         0.12499999 0.         0.         0.12499998 0.
  0.12499999 0.12499999]
 [0.12499998 0.         0.12499999 0.12499999 0.         0.
  0.         0.12499999]
 [0.         0.11180338 0.11180338 0.11180338 0.11180338 0.11180338
  0.         0.        ]
 [0.         0.11180338 0.         0.11180338 0.11180338 0.11180338
  0.         0.11180338]
 [0.         0.12499999 0.12499999 0.         0.         0.
  0.12499999 0.12499999]
 [0.11180338 0.         0.11180338 0.11180338 0.11180338 0.
  0.11180338 0.        ]
 [0.         0.         0.14433755 0.         0.14433755 0.
  0.         0.14433755]
 [0.11180338 0.         0.         0.11180338 0.         0.11180338
  0.11180338 0.11180338]
 [0.12499999 0.         0.         0.12499999 0.         0.12499999
  0.12499999 0.        ]
 [0.10206206 0.10206206 0.         0.         0.10206206 0.10206206
  0.10206206 0.10206206]
 [0.         0.         0.17677668 0.         0.17677668 0.
  0.         0.        ]
 [0.         0.11180338 0

## Testing ThresholdGate

In [126]:
b1 = .2
to_compare = .4
num_qubits = 3
assert b1 <= .5
display(Math(r"\text{Comparing }" + ''.join(str(x) for x in ThresholdGate._bin_repr(b1, num_qubits)) + r"\text{ and }" + ''.join(str(x) for x in ThresholdGate._bin_repr(1 - b1, num_qubits)) + r"\text{ to } |" + ''.join(str(x) for x in ThresholdGate._bin_repr(to_compare, num_qubits)) + r"\rangle"))


if ThresholdGate._bin_repr(b1, num_qubits) == ThresholdGate._bin_repr(to_compare, num_qubits) or ThresholdGate._bin_repr(1 - b1, num_qubits) == ThresholdGate._bin_repr(to_compare, num_qubits):
    display(Math(r"\text{Expected outcome: }|0\rangle"))
elif to_compare < .5:
    if b1 < to_compare:
        display(Math(r"\text{Expected outcome: }|1\rangle"))
    else:
        display(Math(r"\text{Expected outcome: }|0\rangle"))
else:
    if 1 - b1 < to_compare:
        display(Math(r"\text{Expected outcome: }|0\rangle"))
    else:
        display(Math(r"\text{Expected outcome: }|1\rangle"))

qubits = [cirq.LineQubit(i) for i in range(2 * num_qubits + 2)]
circuit = cirq.Circuit()

# Creating a

for index, x in enumerate(ThresholdGate._bin_repr(to_compare, num_qubits)):
    if x:
        circuit.append(cirq.X.on(qubits[index]))

circuit.append(ThresholdGate(b1, num_qubits).on(*qubits))
circuit.append(cirq.measure(qubits[num_qubits], key="outcome"))
visualization_circuit = cirq.Circuit()
visualization_circuit.append(cirq.decompose(circuit[0].operations))
visualization_circuit.append(cirq.decompose(circuit[1].operations))
simulator = cirq.Simulator()
result = simulator.simulate(circuit)
display(Math(r"\text{Outcome: |}" + str(result.measurements["outcome"][0]) + r"\rangle"))
print(visualization_circuit)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

      ┌────┐
0: ─────(0)────@───────────(0)───@───────(0)───@───────────────────(0)─────────
        │      │           │     │       │     │                   │
1: ────X┼──────┼───@───────@─────┼───@───┼─────┼───────────────────┼───────────
        │      │   │       │     │   │   │     │                   │
2: ────X┼──────┼───┼───────┼─────┼───┼───┼─────┼───@───────────────@─────@─────
        │      │   │       │     │   │   │     │   │               │     │
3: ─────┼──────┼───┼───────X─────X───┼───┼─────┼───┼───────────────X─────┼─────
        │      │   │       │     │   │   │     │   │               │     │
4: ─────X──────X───X───@───@─────@───X───X─────X───X─────(0)───@───@─────X─────
                       │                           │     │     │         │
5: ────────────────────X───────────────────────────(0)───@─────X─────────┼─────
                                                         │               │
6: ──────────────────────────────────────────────────────X───────────

In [132]:
x = np.pi/2
[int(x)] + ThresholdGate._bin_repr(x - int(x), 10), x

([1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0], 1.5707963267948966)